### total 12,447개

미국 - 1042개(뉴스)  
미국 - 1094개(블로그)  

여행 - 991개(뉴스)  
여행 - 1098개(블로그)  

네이버 - 815개(뉴스)  
네이버 - 1944개(블로그)  

네이버블로그 - 453개(뉴스)  
네이버블로그 - 975개(블로그)  

네이버쇼핑 - 300개(뉴스)  
네이버쇼핑 - 1000개(블로그)  

네이버지도 - 508개(뉴스)  
네이버지도 - 924개(블로그)  

네이버클라우드 - 376개(뉴스)  
네이버클라우드 - 927개(블로그)  

***import***

In [1]:
import numpy as np
import pandas as pd
import pykospacing
spacer = pykospacing.Spacing()
from konlpy.tag import Okt
okt = Okt()
import re
from gensim import corpora
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

**데이터 불러오기**

In [2]:
NAVERSHOP_news = pd.read_csv('네이버쇼핑(뉴스)-300row.csv')
NAVERSHOP_blog = pd.read_csv('네이버쇼핑(블로그)-1000row.csv')

In [3]:
NAVERSHOP_news

,title,link,description
0,"네이버, 쇼핑라이브로 SME ‘지원사격’…상생 효과 ‘뚜렷’",https://n.news.naver.com/mnews/article/243/000...,"\n\n쇼핑라이브 거래액 절반, SME서 발생정부·지자체 협업 사례도 증가 추세\n..."
1,"네이버 쇼핑라이브, '상생라이브' 지원 확대",https://n.news.naver.com/mnews/article/022/000...,\n\n\t\t\t네이버 쇼핑라이브가 SME의 사업 스케일업을 위한 도구로 자리매김...
2,"네이버 쇼핑라이브, ‘상생라이브’ 지원 확대",https://n.news.naver.com/mnews/article/366/000...,\n\n\n\n\n\n네이버 제공 네이버가 쇼핑라이브가 정부 및 지자체...
3,"네이버·카카오 ""종이→전자문서로 디지털 친환경 앞장서""(종합)",https://n.news.naver.com/mnews/article/031/000...,\n\n5일 환경의 날 맞아 환경 보호 노력 및 성과 공개네이버와 카카오가 5일 환...
4,네이버 '라방'에 청년창업가 등장 확 늘어난다,https://n.news.naver.com/mnews/article/015/000...,\n\n청년 라이브 연 2회→월 2회 확대“온라인 판로·성장 기회 지원”\n\n\n...
...,...,...,...
295,"잇츠스킨, 찐친 크리에이터 서기&주호와 네이버 쇼핑 라이브 진행",https://n.news.naver.com/mnews/article/076/000...,\n\n\t\t\t잇츠한불(대표 김양수)의 클리니컬 솔루션 브랜드 잇츠스킨이 15만...
296,"짝퉁, 나 몰라라?···'억울한' 네이버 오명 씻을까",https://n.news.naver.com/mnews/article/079/000...,\n\n핵심요약국내 중소 패션브랜드들이 네이버와 같은 오픈마켓에서 '짝퉁'이 제재없...
297,"""클릭 한번에 웹툰 캐릭터 완성""…네이버웹툰 AI기술 체험해보니",https://n.news.naver.com/mnews/article/001/001...,"\n\nAI페인터·웹툰미·배경변환 기술 개발 중…""작업 시간 최대 50% 감소 효과..."
298,"쇼핑몰 진단 라플라스, 네이버 커머스솔루션 마켓 입점",https://n.news.naver.com/mnews/article/001/001...,\n\n\n\n\n\n네이버 커머스솔루션 마켓 입점한 라플라스 애널리틱스[라플라스 ...


### 전처리 함수 
*6주차 01.WordCloud 실습코드 참고*

- <span style = 'font-size:1.1em;line-height:1.5em'>(1) 줄바꿈 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(2) 오탈자 교정 (생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(3) 띄어쓰기 교정</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(4) sentence tokenization (너무 오래걸려서 생략)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(5) 특수문자 문자 제거</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(6) word tokenization (형태소 분석 및 stemming)</span>
- <span style = 'font-size:1.1em;line-height:1.5em'>(7) stopword removal</span>

**불용어 목록 생성**

In [4]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords] # stopword 파일의 줄바꿈 문자 제거

In [5]:
def preprocessing(x):
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    results = []
    result = []
    x = x.replace('\n',' ') # (1) 줄바꿈 문자 제거
    df_news = spacer(x) # (3) 띄어쓰기 교정
    
    tokens = okt.morphs(x, stem=True) # (6) word tokenization
    for token in tokens:
        token = p.sub('', token) # (5) 특수문자 제거
        if token not in stopwords:
            result.append(token) # (7) stopword removal
    results.extend(result)
    results = ' '.join(results)
    return results

### pos_taging 함수

#(1) 품사 태깅 수행 

#(2) 명사, 형용사, 부사에 해당하는 단어들만 추출하여 저장

In [6]:
def pos_taging(x):
    pos_news = okt.pos(x) # (1)
    filtered_result = [word for word, pos in pos_news if pos in 'Noun'] # (2)
    nouns_news = [n for n in filtered_result if len(n) > 1]
    return nouns_news

### 전처리 및 postag

In [7]:
NAVERSHOP_news['preprocessed_description'] = NAVERSHOP_news['description'].apply(lambda x: preprocessing(x))
NAVERSHOP_news['preprocessed_description'] = NAVERSHOP_news['preprocessed_description'].apply(lambda x: pos_taging(x))

In [8]:
NAVERSHOP_blog['preprocessed_description'] = NAVERSHOP_blog['description'].apply(lambda x: preprocessing(x))
NAVERSHOP_blog['preprocessed_description'] = NAVERSHOP_blog['preprocessed_description'].apply(lambda x: pos_taging(x))

In [9]:
NAVERSHOP_news['preprocessed_description']

0      [쇼핑, 라이브, 거래, 절반, 발생, 정부, 지자체, 협업, 사례, 증가, 추세,...
1      [네이버, 쇼핑, 라이브, 사업, 스케일, 도구로, 자리, 가운데, 정부, 지자체,...
2      [네이버, 제공, 네이버, 쇼핑, 라이브, 정부, 지자체, 협력, 상생, 라이브, ...
3      [환경, 환경, 보호, 노력, 성과, 공개, 네이버, 카카오, 환경, 환경, 보호,...
4      [청년, 라이브, 확대, 온라인, 판로, 성장, 기회, 지원, 네이버, 쇼핑, 라이...
                             ...                        
295    [잇츠한불, 대표, 김양수, 클리, 솔루션, 브랜드, 잇츠, 스킨, 크리에이터, 서...
296    [핵심, 요약, 국내, 중소, 패션, 브랜드, 네이버, 오픈마켓, 짝퉁, 제재, 유...
297    [페인터, 웹툰, 배경, 변환, 기술, 개발, 작업, 최대, 감소, 효과, 서울, ...
298    [네이버, 커머스, 솔루션, 마켓, 입점, 라플라스, 널리, 라플라스, 제공, 재판...
299    [이데일리, 이윤정, 기자, 글로벌, 푸드, 기업, 로얄, 캐닌, 코리아, 네이버,...
Name: preprocessed_description, Length: 300, dtype: object

### BOW(Bag-of-Words) 생성

In [11]:
word_dict_news = corpora.Dictionary(NAVERSHOP_news['preprocessed_description'])

In [12]:
word_dict_blog = corpora.Dictionary(NAVERSHOP_blog['preprocessed_description'])

In [13]:
corpus_news = [word_dict_news.doc2bow(text) for text in NAVERSHOP_news['preprocessed_description']]

In [14]:
corpus_blog = [word_dict_blog.doc2bow(text) for text in NAVERSHOP_blog['preprocessed_description']]

In [15]:
corpus_news[2]

[(0, 2),
 (1, 1),
 (2, 2),
 (5, 1),
 (12, 1),
 (13, 4),
 (15, 2),
 (17, 7),
 (22, 1),
 (23, 1),
 (25, 1),
 (27, 15),
 (29, 1),
 (32, 2),
 (33, 1),
 (36, 1),
 (40, 1),
 (42, 1),
 (45, 1),
 (46, 2),
 (48, 1),
 (49, 1),
 (52, 1),
 (53, 1),
 (56, 2),
 (57, 1),
 (60, 2),
 (61, 1),
 (62, 6),
 (64, 1),
 (67, 1),
 (68, 1),
 (71, 2),
 (73, 1),
 (74, 1),
 (77, 1),
 (82, 1),
 (84, 3),
 (85, 1),
 (87, 3),
 (90, 1),
 (94, 2),
 (102, 1),
 (103, 1),
 (105, 1),
 (107, 1),
 (108, 1),
 (109, 1),
 (111, 2),
 (112, 4),
 (113, 3),
 (114, 1),
 (115, 1),
 (116, 1),
 (124, 3),
 (125, 5),
 (126, 2),
 (127, 1),
 (130, 2),
 (131, 1),
 (132, 3),
 (133, 1),
 (134, 1),
 (135, 4),
 (136, 1),
 (137, 1),
 (139, 1),
 (141, 1),
 (142, 1),
 (144, 1),
 (145, 2),
 (147, 1),
 (150, 3),
 (151, 1),
 (156, 1),
 (157, 1),
 (159, 1),
 (162, 1),
 (163, 1),
 (165, 3),
 (166, 1),
 (169, 1),
 (170, 1),
 (173, 1),
 (176, 1),
 (178, 1),
 (179, 1),
 (189, 1),
 (192, 1),
 (204, 1),
 (206, 1)]

In [16]:
[(idx,word) for idx, word in word_dict_news.items()]

[(0, '가치'),
 (1, '개설'),
 (2, '거래'),
 (3, '결과'),
 (4, '경우'),
 (5, '경험'),
 (6, '공인'),
 (7, '관련'),
 (8, '교구장'),
 (9, '교육'),
 (10, '금요일'),
 (11, '기관'),
 (12, '기능'),
 (13, '기술'),
 (14, '기자'),
 (15, '기회'),
 (16, '기획전'),
 (17, '네이버'),
 (18, '노출'),
 (19, '단골'),
 (20, '대비'),
 (21, '데이즈'),
 (22, '동영상'),
 (23, '동행'),
 (24, '등급'),
 (25, '디지털'),
 (26, '라며'),
 (27, '라이브'),
 (28, '러브콜'),
 (29, '리더'),
 (30, '마케팅'),
 (31, '매달'),
 (32, '매월'),
 (33, '매출'),
 (34, '명과'),
 (35, '모델'),
 (36, '무상'),
 (37, '발생'),
 (38, '방증'),
 (39, '배송비'),
 (40, '벤처기업'),
 (41, '블루'),
 (42, '비즈니스'),
 (43, '빈도'),
 (44, '사례'),
 (45, '사업'),
 (46, '상생'),
 (47, '상승'),
 (48, '상시'),
 (49, '상품'),
 (50, '새싹'),
 (51, '생산'),
 (52, '생산자'),
 (53, '생태계'),
 (54, '서비스'),
 (55, '선정'),
 (56, '성장'),
 (57, '센터'),
 (58, '소개'),
 (59, '소상'),
 (60, '솔루션'),
 (61, '송재훈'),
 (62, '쇼핑'),
 (63, '수산물'),
 (64, '수요일'),
 (65, '수준'),
 (66, '순항'),
 (67, '스마트'),
 (68, '스마트폰'),
 (69, '스쿨'),
 (70, '스퀘어'),
 (71, '스토어'),
 (72, '스튜디오'),
 (73, '시스템'),
 (74, '시장'),
 (75,

In [17]:
[(idx,word) for idx, word in word_dict_blog.items()]

[(0, '가루'),
 (1, '가시'),
 (2, '가족'),
 (3, '간장'),
 (4, '감동'),
 (5, '강화'),
 (6, '개월'),
 (7, '개인'),
 (8, '걱정'),
 (9, '검색'),
 (10, '경험'),
 (11, '계란'),
 (12, '고니'),
 (13, '고명'),
 (14, '고민'),
 (15, '고춧가루'),
 (16, '구매'),
 (17, '구성은'),
 (18, '국물'),
 (19, '궁합'),
 (20, '그릇'),
 (21, '금액'),
 (22, '기대'),
 (23, '기준'),
 (24, '기한'),
 (25, '김치'),
 (26, '깍두기'),
 (27, '납짝'),
 (28, '냄비'),
 (29, '냉동'),
 (30, '냉장'),
 (31, '네이버'),
 (32, '느낌'),
 (33, '다르쥬'),
 (34, '다시'),
 (35, '다행'),
 (36, '대로'),
 (37, '대중'),
 (38, '대파'),
 (39, '덕분'),
 (40, '도착'),
 (41, '들깨'),
 (42, '라면'),
 (43, '레시피'),
 (44, '리다'),
 (45, '리릭'),
 (46, '만요'),
 (47, '매장'),
 (48, '먹기'),
 (49, '면빨'),
 (50, '물이'),
 (51, '뭔가'),
 (52, '미리'),
 (53, '밀가루'),
 (54, '바닥'),
 (55, '박스'),
 (56, '반찬'),
 (57, '방문'),
 (58, '방법'),
 (59, '배려'),
 (60, '배송'),
 (61, '베레'),
 (62, '베테'),
 (63, '보관'),
 (64, '보기'),
 (65, '보습'),
 (66, '비닐'),
 (67, '비법'),
 (68, '비쥬'),
 (69, '사용'),
 (70, '살짝'),
 (71, '생각'),
 (72, '선호'),
 (73, '쇼핑'),
 (74, '순무'),
 (75, '스티로폼'),
 (76, '아이스박스

In [18]:
len(word_dict_news)

5116

### LDA 모델 훈련

#(1) 3개의 topic

#(2) LDA 모델을 생성. 'id2word'에는 단어 사전을 전달하고, 학습 반복 횟수는 15로 설정

#(3) 생성된 모델에서 각 주제에 대한 상위 4개 단어를 출력


In [27]:
N_TOPICS = 3 # (1) 
ldamodel_news = gensim.models.ldamodel.LdaModel(corpus_news, num_topics = N_TOPICS, id2word=word_dict_news, passes = 15) # (2)

topics = ldamodel_news.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.019*"네이버" + 0.011*"라이브" + 0.011*"상품" + 0.010*"판매"')
(1, '0.040*"네이버" + 0.015*"서비스" + 0.014*"라이브" + 0.011*"쇼핑"')
(2, '0.031*"네이버" + 0.021*"쇼핑" + 0.019*"메인" + 0.018*"모바일"')


In [28]:
N_TOPICS = 3 # (1)
ldamodel_blog = gensim.models.ldamodel.LdaModel(corpus_blog, num_topics = N_TOPICS, id2word=word_dict_blog, passes = 15) # (2)

topics = ldamodel_blog.print_topics(num_words=4) # (3)
for topic in topics:
    print(topic)

(0, '0.053*"라이브" + 0.029*"쇼핑" + 0.024*"네이버" + 0.023*"방송"')
(1, '0.032*"네이버" + 0.019*"쇼핑" + 0.011*"사용" + 0.009*"구매"')
(2, '0.030*"쇼핑" + 0.025*"네이버" + 0.023*"라이브" + 0.017*"광고"')


### LDA 시각화

In [29]:
vis_news = pyLDAvis.gensim_models.prepare(ldamodel_news, corpus_news, word_dict_news)
pyLDAvis.display(vis_news)

In [30]:
vis_blog = pyLDAvis.gensim_models.prepare(ldamodel_blog, corpus_blog, word_dict_blog)
pyLDAvis.display(vis_blog)